In [12]:
import cPickle as pickle
import numpy as np
from pprint import pprint

with open('thesis/results/features_2016-12-14_10-15-43.pkl') as f:
    data, configuration = pickle.load(f)

print "PROJECTS:"
for p_name in data.iterkeys():
    print p_name

print
print "CONFIG:"
pprint(configuration)

PROJECTS:
Sowbug3
Cam1
Camera3
Zebrafish

CONFIG:
{'rf_class_weight': 'balanced_subsample',
 'rf_criterion': 'entropy',
 'seed': 42,
 'test_split_method': 'random'}


In [22]:
p_names = filter(lambda x: x != 'layer', [p for p in data.iterkeys()])
print p_names
splits = filter(lambda x: x != 'layer', [s for s in data[p_names[0]].iterkeys()])
print splits
features = filter(lambda x: x!= 'layer', [f for f in data[p_names[0]][splits[0]].iterkeys()])
print features

# for p_name, data in data.iteritems():
#     print p_name

['Sowbug3', 'Cam1', 'Camera3', 'Zebrafish']
[0.95, 0.9, 0.8, 0.99]
['fm_lbp ', 'fm_colornames ', 'fm_idtracker_c_d50 ', 'fm_idtracker_i_d50 ', 'fm_basic ', 'fm_hog ']


In [100]:
from tabulate import tabulate

splits = sorted(splits)
for split in splits:
    # print "------- ", split, " -------"
    
    make_bold = []
    
    # +2 - caption + dataset size
    table = [[] for _ in range(len(features) + 2)]
    i = 0
    table[i].append('')
    for p_name in p_names:
        table[i].append(p_name)
        make_bold.append(p_name)
    
    i += 1
    table[i].append('')
    for p_name in p_names:
        table[i].append(data[p_name][split][features[0]]['X_shape'][0])
        
    keys = []
    
    sample_sizes = []
    
    for feat in features:
        i += 1
        # remove fm_ p
        f = '_'.join(feat.split('_')[1:])
        
        sample_sizes.append(data[p_name][split][feat]['X_shape'][0])
        
        if f[:6] == 'idtrac':
            f = f[:-5]
            if f[-1] == 'c':
                f = f[:-2]+' C'
            else:
                f = f[:-2]+' I'
        
        table[i].append(f)
        make_bold.append(f)

        for p_name in p_names:
            it = data[p_name][split][feat]
            if isinstance(it, dict):
                std = "{:.3f}".format(np.std(it['accuracy']))
                s = "{:.2}".format(np.mean(it['accuracy'])) 
                s += " ("+std[1:]+")"
                table[i].append(s)
                
    
    print
    from math import ceil
    num_runs = len(data[p_name][split][feat]['class_frequency'])
    num_regions = np.mean(sample_sizes)
    num_train = int((1-split)*num_regions)
    
    title = "Features performance, trained on {}\% (\#regions displayed under dataset names), randomly picked".format(int(100 - (100*split)))
    label = 'features_accuracy_'+str(int(split*100))
    
    s = tabulate(table, tablefmt='latex')
    pre = '\\begin {table}[H] \\caption {'+title+'} \\label{tab:'+label+'} \\begin{center}'
    post = '\\end{center}\\end {table}'
    for t in make_bold:
        s = s.replace(t, '\\textbf{'+t+'}')
    s = pre + s + post
    print s
    print


\begin {table}[H] \caption {Features performance, trained on 20\% (\#regions displayed under dataset names), randomly picked} \label{tab:features_accuracy_80} \begin{center}\begin{tabular}{lllll}
\hline
             & \textbf{Sowbug3}     & \textbf{Cam1}        & \textbf{Camera3}     & \textbf{Zebrafish}   \\
             & 8386        & 7639        & 11701       & 20925       \\
 \textbf{lbp }        & 0.7 (.005)  & 0.52 (.007) & 0.4 (.007)  & 0.39 (.003) \\
 \textbf{colornames } & 0.82 (.007) & 0.79 (.008) & 0.7 (.004)  & 0.59 (.004) \\
 \textbf{idtracker C} & 0.79 (.005) & 0.89 (.004) & 0.82 (.006) & 0.82 (.004) \\
 \textbf{idtracker I} & 0.82 (.006) & 0.92 (.003) & 0.85 (.007) & 0.88 (.004) \\
 \textbf{basic }      & 0.87 (.004) & 0.79 (.004) & 0.64 (.003) & 0.68 (.004) \\
 \textbf{hog }        & 0.69 (.007) & 0.51 (.006) & 0.4 (.004)  & 0.39 (.002) \\
\hline
\end{tabular}\end{center}\end {table}


\begin {table}[H] \caption {Features performance, trained on 10\% (\#regions displa